In [1]:
import pandas as pd 
import os
import json
import mne

os.chdir('..')

In [3]:
subject_id = 12
trial_id = 1

properties_path = f'Datasets/Data/S_{subject_id}/Trial_{trial_id}/EEG_Properties.json'
datablocks_path = f'Datasets/Data/S_{subject_id}/Trial_{trial_id}/EEG.csv'
eyetracker_path = f'Datasets/Data/S_{subject_id}/Trial_{trial_id}/Eyetracker.asc'
experiment_path = f'Datasets/Data/S_{subject_id}/Trial_{trial_id}/Experiment.json'

with open(properties_path,'r') as f:
    props = json.load(f)
sr = props['sampling_interval']
chan_names  = props['channel_names']

with open(experiment_path,'r') as f:
    experiment_seq = json.load(f)

eye_raw = mne.io.read_raw_eyelink(eyetracker_path, verbose=False)
eye_len = int(len(eye_raw) / sr)

eeg_df = pd.read_csv(datablocks_path, header=None, names=['block_id']+chan_names)
eeg_df = eeg_df.replace(r';', '', regex=True).astype(float)
eeg_duration = int(len(eeg_df) / sr)

expected_duration = 1557   # Adapted impirecally from experiment
block_duration = sum(block['content'].get('duration') for block in experiment_seq.values())

print(f"Expected: \t {expected_duration} sec. \t {expected_duration / 60:.2f} min. \n-------------------------------------------------------\nGot Blocks: \t {block_duration} sec. \t {block_duration / 60:.2f} min.\nGot EEG: \t {int(eeg_duration)} sec. \t {eeg_duration / 60:.2f} min.\nGot Eyetracker:  {eye_len} sec. \t {eye_len / 60:.2f} min.")


Expected: 	 1557 sec. 	 25.95 min. 
-------------------------------------------------------
Got Blocks: 	 1548 sec. 	 25.80 min.
Got EEG: 	 1568 sec. 	 26.13 min.
Got Eyetracker:  1568 sec. 	 26.13 min.
